# scFair tutorial

# install

In [ ]:
# required packages (run in terminal)
pip install torch torchvision torchaudio
pip install scvi-tools
pip install matplotlib
pip install scanpy
pip install scikit-misc
pip install xgboost
pip install fairlearn

In [ ]:
# install scfair (run in terminal)
pip install git+https://github.com/M0hammadL/scfair.git

# clone scfair-reproducibility (run in terminal)
git clone https://github.com/M0hammadL/scfair-reproducibility.git
# then rename the cloned directory to: scfair_reproducibility

# import packages

In [17]:
# enable autoreload
%load_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext autoreload

import scvi
scvi.settings.seed = 0
import scanpy as sc
import anndata as ad
import torch
import numpy as np
import pandas as pd
from datetime import datetime
from scipy.sparse import csr_matrix
torch.set_float32_matmul_precision('medium')
import warnings
warnings.simplefilter("ignore", UserWarning)
import sys


Global seed set to 0


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# check if you are in a child of the project directory, and if so, change to the main directory: scfair-reproducibility.

In [21]:
import os

!pwd
path_current = os.getcwd()
print(path_current.split('/')[-1])
if path_current.split('/')[-1]=="scfair-reproducibility":
    print("we are in the correct directory")
else:
    os.chdir("..")
    print("we are changing directory to")
    !pwd
    
    
from evaluation.metrics import *
from evaluation.evaluate import *
from scib_metrics_dev.src.scib_metrics.benchmark import Benchmarker

/home/jupyter/analysis/scfair_analysis/scfair-reproducibility/notebooks
/home/jupyter/analysis/scfair_analysis/scfair-reproducibility


# prepare data

In [24]:
# load dataset
adata = scvi.data.heart_cell_atlas_subsampled()

# preprocess dataset
sc.pp.filter_genes(adata, min_counts=3)
adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=1200,
    subset=True,
    layer="counts",
    flavor="seurat_v3",
)

# specify name of dataset 
data_name = 'heartAtlas'

# specify attributes
cats = ['cell_type', 'cell_source', 'gender', 'region']

# specify a path that will be used to save any trained model later (directories in the path should be created first)
pre_path = f"models/FairVI"

# specify a path that will be used to save the preprocessed indices of paired samples for the counterfactual term calculation
idx_cf_tensor_path = f'idx_cf_tensors/heart_4cov'

# create numerical index for each attr in cats
create_cats_idx(adata, cats)

today = datetime.today().strftime('%Y-%m-%d')

adata

INFO     File data/hca_subsampled_20k.h5ad already downloaded                                                      


AnnData object with n_obs × n_vars = 18641 × 1200
    obs: 'NRP', 'age_group', 'cell_source', 'cell_type', 'donor', 'gender', 'n_counts', 'n_genes', 'percent_mito', 'percent_ribo', 'region', 'sample', 'scrublet_score', 'source', 'type', 'version', 'cell_states', 'Used', 'cell_type_idx', 'cell_source_idx', 'gender_idx', 'region_idx'
    var: 'gene_ids-Harvard-Nuclei', 'feature_types-Harvard-Nuclei', 'gene_ids-Sanger-Nuclei', 'feature_types-Sanger-Nuclei', 'gene_ids-Sanger-Cells', 'feature_types-Sanger-Cells', 'gene_ids-Sanger-CD45', 'feature_types-Sanger-CD45', 'n_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'cell_type_colors', 'log1p', 'hvg'
    layers: 'counts'

# create directory for counterfactuals, named "idx_cf_tensors"


In [27]:
# create directory with counterfactuals
isExist = os.path.exists(idx_cf_tensor_path)
if not isExist:
    os.makedirs(idx_cf_tensor_path)
    print("The new directory is created!")
else:
    print("The directory already exists!")

# create directory with models
isExist = os.path.exists("models")
if not isExist:
    os.makedirs(idx_cf_tensor_path)
    print("The models directory is created!")
else:
    print("The models directory already exists!")
   

The directory already exists!
The models directory already exists!


# load/train model

In [ ]:
# specify train parameters
epochs = 400
batch_size = 256
cf_weight = 2
alpha = 1
clf_weight = 50
adv_clf_weight = 10
adv_period = 1
mode=(0,1,2,3,4)

train_dict = {'max_epochs': epochs, 'batch_size': batch_size, 'cf_weight': cf_weight,
              'alpha': alpha, 'clf_weight': clf_weight, 'adv_clf_weight': adv_clf_weight,
              'adv_period': adv_period, 'mode': mode}

# specify a name for your model
model_name = today + ',' + data_name + ',' + ','.join(cats) + ',' + ','.join(k + '=' + str(v) for k, v in train_dict.items())

# load model (if trained before)
try:
    model = FairVI.load(f"{pre_path}/{model_name}", adata=adata)

# trains the model (if not trained before) and save it into: pre_path + model_name
except:

    FairVI.setup_anndata(
        adata,
        layer='counts',
        categorical_covariate_keys=cats,
        continuous_covariate_keys=[]
    )
    model = FairVI(adata, idx_cf_tensor_path=idx_cf_tensor_path)
    model.train(**train_dict)
    model.save(f"{pre_path}/{model_name}")

model.idx_cf_tensor_path = idx_cf_tensor_path

INFO     No backup URL provided for missing file                                                                   
         models/FairVI/2023-08-25,heartAtlas,cell_type,cell_source,gender,region,max_epochs=400,batch_size=256,cf_w
         eight=2,alpha=1,clf_weight=50,adv_clf_weight=10,adv_period=1,mode=(0, 1, 2, 3, 4)/model.pt                


No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/jupyter/analysis/scfair_analysis/scfair_analysis_env/lib/python3.10/site-packages/lightning/pytorch/core/optimizer.py:321: RuntimeWarning: The lr scheduler dict contains the key(s) ['monitor'], but the keys will be ignored. You need to call `lr_scheduler.step()` manually in manual optimization.
  rank_zero_warn(


Epoch 30/400:   7%|▋         | 29/400 [07:29<1:35:11, 15.39s/it, v_num=1, loss_validation=4.19e+3, x_0_validation=290, x_1_validation=298, x_2_validation=298, x_3_validation=298, x_4_validation=300, xcf_1_validation=316, xcf_2_validation=322, xcf_3_validation=316, xcf_4_validation=309, z_1_validation=20.6, z_2_validation=19.8, z_3_validation=20.9, z_4_validation=19.6, ce_validation=0.912, acc_validation=0.999, f1_validation=0.999, adv_ce_validation=1.33, adv_acc_validation=0.573, adv_f1_validation=0.573, loss_train=3.98e+3, x_0_train=280, x_1_train=291, x_2_train=291, x_3_train=290, x_4_train=292, xcf_1_train=298, xcf_2_train=304, xcf_3_train=299, xcf_4_train=296, z_1_train=21.4, z_2_train=20.6, z_3_train=21.5, z_4_train=20.1, ce_train=0.913, acc_train=0.999, f1_train=0.999, adv_ce_train=1.25, adv_acc_train=0.556, adv_f1_train=0.556]

# latent space

In [ ]:
# trains the model (if not trained before) and save it into: pre_path + model_name
# then get latent representaion (they will be strored in adata.obsm)
# Z_0 = adata.obsm["Z_0"]
# Z_i =  adata.obsm["Z_i"] for i in [1, ..., len(cats)]
# Z_{-i} = adata.obsm["Z _not_i"] for i in [1, ..., len(cats)]
model, adata = latent(
        adata = adata,
        cats = cats,
        new_model_name = model_name,
        pre_path = pre_path,
        idx_cf_tensor_path = idx_cf_tensor_path,
        plot_umap = True,
        **train_dict,
)

# Disentanglement/Fainess metrics

In [7]:
# classifier Si | Zi
acc_results_1 = clf_S_Z_metrics(adata, cats)
# classifier Si | (Z - Zi)
acc_results_2 = clf_S_Z_not_metrics(adata, cats)

# fairness metrics: DP, EO, ...
create_cats_idx(adata, ['NRP'])
y_name = 'NRP_idx'
ACC, DP_diff, EO_diff = fair_clf_metrics(adata, cats, y_name)

# Max Mutual Information by taking Max over Dims
MI_md, MI_not_md, mig_md = max_dim_MI_metrics(adata, cats)
# Mutual Information by Mixed_KSG (https://github.com/wgao9/mixed_KSG/blob/master/mixed.py)
MI, MI_not, MI_not_max, mig, mipg = Mixed_KSG_MI_metrics(adata, cats)

metrics for XGBoost classifier Si | Zi
train acc S1: 1.0000,  test acc S1: 0.9987
train acc S2: 1.0000,  test acc S2: 0.9991
train acc S3: 1.0000,  test acc S3: 0.9995
train acc S4: 1.0000,  test acc S4: 0.9979
metrics for XGBoost classifier Si | (Z - Zi)
train acc S1: 0.9985,  test acc S1: 0.7681
train acc S2: 0.9903,  test acc S2: 0.7600
train acc S3: 0.9402,  test acc S3: 0.7289
train acc S4: 0.9658,  test acc S4: 0.4732
fairness metrics wrt Si for XGBoost classifier NRP_idx_bin | (Z - Zi)
i=1: accuracy = 0.9666, DP_diff = 0.0000, EO_diff = 0.0000
i=2: accuracy = 0.8869, DP_diff = 0.0000, EO_diff = 1.0000
i=3: accuracy = 0.9006, DP_diff = 0.0000, EO_diff = 0.0000
i=4: accuracy = 0.9649, DP_diff = 0.0000, EO_diff = 0.0000
Max_Dim Mutual Information metrics
MI(Z_1 ; S_1) = 1.0728,  MI((Z - Z_1) ; S_1) = 0.3250
MI(Z_2 ; S_2) = 1.1210,  MI((Z - Z_2) ; S_2) = 0.2348
MI(Z_3 ; S_3) = 0.6677,  MI((Z - Z_3) ; S_3) = 0.0262
MI(Z_4 ; S_4) = 0.9175,  MI((Z - Z_4) ; S_4) = 0.0759
MIG = 0.6276
Mi

# OOD metrics

In [8]:
cov_idx = 2  # index of target attribute in cats
cov_value = 'Male'  # factual value for the target attribute
cov_value_cf = 'Female'  # counterfactual value for the target attribute
other_covs_values = ('Ventricular_Cardiomyocyte', 'Sanger-Nuclei', 'RV')  # fixed values for other attibutes that we perform OOD on them
n_top_deg = 100  # number of top DE genes for R2 

# holds-out all cells with other_covs_values (prefered method for OOD)
# splits other cells to train/validation sets, trains the model, and finally perform OOD on held-out cells
# returns: 
#     1) true genes counts vector by averaging all held-out cells
#     2) predicted genes counts vector for average of held-out cells
true_x_counts_mean, px_cf_mean_pred = ood_for_given_covs_2(
        adata=adata,
        cats=cats,
        new_model_name=model_name,
        pre_path=pre_path,
        idx_cf_tensor_path=idx_cf_tensor_path,
        cov_idx=cov_idx,
        cov_value=cov_value,
        cov_value_cf=cov_value_cf,
        other_covs_values=other_covs_values,
        n_top_deg=n_top_deg,
        **train_dict,
)

INFO     No backup URL provided for missing file                                                                   
         models/FairVI/2023-08-22T18:12:51.733765,heartAtlas,cell_type,cell_source,gender,region,max_epochs=400,bat
         ch_size=256,cf_weight=2,alpha=1,clf_weight=50,adv_clf_weight=10,adv_period=1,mode=(0, 1, 2, 3,            
         4)--cf--gender = Male to Female, cell_type = Ventricular_Cardiomyocyte, cell_source = Sanger-Nuclei,      
         region = RV/model.pt                                                                                      


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [MIG-GPU-18bbdf05-c7ca-a9f7-0896-7641c359d553/0/0]
/lustre/scratch126/cellgen/team205/ks32/conda/lib/python3.10/site-packages/lightning/pytorch/core/optimizer.py:321: RuntimeWarning: The lr scheduler dict contains the key(s) ['monitor'], but the keys will be ignored. You need to call `lr_scheduler.step()` manually in manual optimization.
  rank_zero_warn(


Epoch 62/400:  16%|█▌        | 62/400 [09:49<53:32,  9.50s/it, v_num=1, loss_validation=4.17e+3, x_0_validation=285, x_1_validation=292, x_2_validation=293, x_3_validation=294, x_4_validation=295, xcf_1_validation=315, xcf_2_validation=314, xcf_3_validation=317, xcf_4_validation=312, z_1_validation=18.7, z_2_validation=17.5, z_3_validation=17.7, z_4_validation=17.2, ce_validation=0.913, acc_validation=0.998, f1_validation=0.998, adv_ce_validation=1.32, adv_acc_validation=0.587, adv_f1_validation=0.587, loss_train=3.97e+3, x_0_train=281, x_1_train=291, x_2_train=291, x_3_train=292, x_4_train=293, xcf_1_train=296, xcf_2_train=301, xcf_3_train=296, xcf_4_train=293, z_1_train=19.3, z_2_train=18.6, z_3_train=18.5, z_4_train=17.9, ce_train=0.912, acc_train=0.999, f1_train=0.999, adv_ce_train=1.14, adv_acc_train=0.56, adv_f1_train=0.56]  
Monitored metric loss_validation did not improve in the last 45 records. Best score: 4131.867. Signaling Trainer to stop.
INFO     AnnData object appears to

In [ ]:
# if you like to hold-out only cells with other_covs_values and cov_value_cf use:
true_x_counts_mean, px_cf_mean_pred = ood_for_given_covs_1(
        adata=adata,
        cats=cats,
        new_model_name=model_name,
        pre_path=pre_path,
        idx_cf_tensor_path=idx_cf_tensor_path,
        cov_idx=cov_idx,
        cov_value=cov_value,
        cov_value_cf=cov_value_cf,
        other_covs_values=other_covs_values,
        n_top_deg=n_top_deg,
        **train_dict,
)

In [10]:
# R2 metrics (both ood_for_given_covs methods above use it)
# you can try different values of n_top_deg
r2_eval(adata, true_x_counts_mean, px_cf_mean_pred, n_top_deg=20)

All Genes
R2 = 0.8956
R2 log = 0.8955
DE Genes (n_top=20)
R2 = 0.9195
R2 log = 0.9337
